<a href="https://colab.research.google.com/github/yukilost/googlecolab/blob/main/LearningBasedFluids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>GraphNeruralNetworkを用いた非圧縮性流体の教師なし学習</h1>

TODO
- Encoder
- Decoder
- Processor
- NNS
- Train
- Valid
- Test
- Refactor

MEMO
- 基本的にはlistを使用
- ベクトル演算などはndarray

#初期設定

##ライブラリのインポート

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn import neighbors
import torch

##GoogleDriveのマウント

In [8]:
drive.mount("/content/drive")

Mounted at /content/drive


##デバイス設定

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


##定数

#データセット

In [9]:
!unzip /content/drive/MyDrive/LearningBasedFluids/data.zip -d /content/graph_neural_network/

Archive:  /content/drive/MyDrive/LearningBasedFluids/data.zip
   creating: /content/graph_neural_network/data/
   creating: /content/graph_neural_network/data/20/
  inflating: /content/graph_neural_network/data/20/20.json  
  inflating: /content/graph_neural_network/data/20/20.csv  
   creating: /content/graph_neural_network/data/18/
  inflating: /content/graph_neural_network/data/18/18.csv  
  inflating: /content/graph_neural_network/data/18/18.json  
   creating: /content/graph_neural_network/data/27/
  inflating: /content/graph_neural_network/data/27/27.json  
  inflating: /content/graph_neural_network/data/27/27.csv  
   creating: /content/graph_neural_network/data/9/
  inflating: /content/graph_neural_network/data/9/9.csv  
  inflating: /content/graph_neural_network/data/9/9.json  
   creating: /content/graph_neural_network/data/11/
  inflating: /content/graph_neural_network/data/11/11.json  
  inflating: /content/graph_neural_network/data/11/11.csv  
   creating: /content/graph_n

In [10]:
df = pd.read_csv("graph_neural_network/data/1/1.csv")
df

,idx,timestep,label,volume,mass,density,position.x,position.y,position.z,velocity.x,velocity.y,velocity.z
0,0,1,0,0.000065,0.062089,899.017,0.017109,0.017109,0.017109,0.000000,0.000000,0.000000
1,1,1,0,0.000056,0.053663,974.240,0.017109,0.017109,0.051329,0.000000,0.000000,0.000000
2,2,1,0,0.000056,0.053663,957.703,0.017109,0.017109,0.085548,0.000000,0.000000,0.000000
3,3,1,0,0.000056,0.053663,957.957,0.017109,0.017109,0.119767,0.000000,0.000000,0.000000
4,4,1,0,0.000056,0.053663,957.957,0.017109,0.017109,0.153986,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
791995,3163,250,1,0.000042,0.040000,1028.830,0.608688,0.085820,0.351352,0.042495,0.019862,0.054630
791996,3164,250,1,0.000042,0.040000,956.616,0.599913,0.119947,0.355292,-0.028329,0.028179,0.050579
791997,3165,250,1,0.000042,0.040000,1085.280,0.575361,0.051333,0.549272,0.041591,-0.000683,0.025245
791998,3166,250,1,0.000042,0.040000,970.343,0.562462,0.116581,0.490510,0.054284,0.032399,0.002483


In [17]:
config = pd.read_json("graph_neural_network/data/1/1.json")
config

,Camera,Light,OpenGL,PBF,Shader,Simulation,Window
ClippingFar,50,NaN,NaN,NaN,NaN,NaN,NaN
ClippingNear,0.1,NaN,NaN,NaN,NaN,NaN,NaN
FieldOfView,45,NaN,NaN,NaN,NaN,NaN,NaN
Position,"{'x': 1.0, 'y': 1.5, 'z': 2.0}",NaN,NaN,NaN,NaN,NaN,NaN
Target,"{'x': 0.0, 'y': 0.0, 'z': 0.0}",NaN,NaN,NaN,NaN,NaN,NaN
WorldUp,"{'x': 0.0, 'y': 1.0, 'z': 0.0}",NaN,NaN,NaN,NaN,NaN,NaN
Color,NaN,"{'x': 1.0, 'y': 1.0, 'z': 1.0}",NaN,NaN,NaN,NaN,NaN
Direction,NaN,"{'x': 1.0, 'y': 1.0, 'z': 1.0}",NaN,NaN,NaN,NaN,NaN
IsCoreProfile,NaN,NaN,1.0,NaN,NaN,NaN,NaN
VersionMejor,NaN,NaN,3.0,NaN,NaN,NaN,NaN


In [12]:
df.keys()

Index(['idx', 'timestep', 'label', 'volume', 'mass', 'density', 'position.x',
       'position.y', 'position.z', 'velocity.x', 'velocity.y', 'velocity.z'],
      dtype='object')

In [13]:
df["volume"]

0         0.000065
1         0.000056
2         0.000056
3         0.000056
4         0.000056
            ...   
791995    0.000042
791996    0.000042
791997    0.000042
791998    0.000042
791999    0.000042
Name: volume, Length: 792000, dtype: float64

#グラフニューラルネットワーク

In [14]:
df[df["timestep"] == 1]

,idx,timestep,label,volume,mass,density,position.x,position.y,position.z,velocity.x,velocity.y,velocity.z
0,0,1,0,0.000065,0.062089,899.017,0.017109,0.017109,0.017109,0.000000,0.000000,0.000000
1,1,1,0,0.000056,0.053663,974.240,0.017109,0.017109,0.051329,0.000000,0.000000,0.000000
2,2,1,0,0.000056,0.053663,957.703,0.017109,0.017109,0.085548,0.000000,0.000000,0.000000
3,3,1,0,0.000056,0.053663,957.957,0.017109,0.017109,0.119767,0.000000,0.000000,0.000000
4,4,1,0,0.000056,0.053663,957.957,0.017109,0.017109,0.153986,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
3163,3163,1,1,0.000042,0.040000,751.114,0.622365,0.560582,0.258867,-0.219984,-0.255265,0.412969
3164,3164,1,1,0.000042,0.040000,751.114,0.622365,0.560582,0.293086,-0.219984,-0.255265,0.412969
3165,3165,1,1,0.000042,0.040000,751.114,0.622365,0.560582,0.327305,-0.219984,-0.255265,0.412969
3166,3166,1,1,0.000042,0.040000,751.114,0.622365,0.560582,0.361524,-0.219984,-0.255265,0.412969


In [15]:
df[df["timestep"] == 1][["position.x", "position.y", "position.z"]].values

array([[0.0171095, 0.0171095, 0.0171095],
       [0.0171095, 0.0171095, 0.0513286],
       [0.0171095, 0.0171095, 0.0855476],
       ...,
       [0.622365 , 0.560582 , 0.327305 ],
       [0.622365 , 0.560582 , 0.361524 ],
       [0.622365 , 0.560582 , 0.395743 ]])

In [19]:
# TODO: leaf_sizeについて調べる
# TODO: ハッシュ値を使った実装に変更

def NearestNeighborSearch(pos, radius):
  tree = neighbors.KDTree(pos, leaf_size=2)
  idx = tree.query_radius(pos[:], r=radius)
  return idx

In [15]:
pos = df[df["timestep"] == 1][[" position.x", " position.y", " position.z"]].values
idx = NearestNeighborSearch(pos, 0.05)
idx

array([array([  0,   1, 401, 400,  21,  20, 420]),
       array([  0,   1, 401, 400,   2, 402,  21,  20,  22]),
       array([  1, 401,   2,   3, 403, 402,  21,  23,  22]), ...,
       array([3156, 3164, 3154, 3064, 3155, 3055, 3065, 3166, 3066, 3165]),
       array([3056, 3156, 3155, 3065, 3166, 3066, 3165, 3157, 3167, 3067]),
       array([3156, 3166, 3066, 3057, 3157, 3167, 3067])], dtype=object)

In [52]:
def Encode(df, radius):
  V = df[["position.x", "position.y", "position.z", "velocity.x", "velocity.y", "velocity.z"]].values

  neighbor_indices = NearestNeighborSearch(df[["position.x", "position.y", "position.z"]].values, radius)

  receivers = np.zeros()
  senders = np.zeros()
  relative_positions = np.zeros()
  for receiver, neighbor in enumerate(neighbor_indices):
    for sender in neighbor:
      # 近傍粒子から自身を除く
      if sender == receiver: continue
      receivers = np.append(receivers, [receiver])
      snders = np.append(senders, [sender])
      r_ij = df[sender:sender+1][["position.x", "position.y", "position.z"]].values - df[receiver:receiver+1][["position.x", "position.y", "position.z"]].values
      E = np.append(E, [r_ij, receiver, sender])
  return [V, E]

In [81]:
tmp = np.zeros(0)
tmp = np.append(tmp, [np.array([1, 2]),])
tmp

array([1., 2., 2., 3.])

In [53]:
V0, E0 = Encode(df[df["timestep"] == 1], 0.05)
E0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


ValueError: ignored